In [7]:
import os
#import utils
import pandas as pd

In [ ]:
def pebba(file_in, 
          gmt_file, 
          gene_col="Gene.symbol",
          logFC_col="logFC",
          pvalue_col="P.Value",
          min_genes=100,
          max_genes=1500,
          p_cut=0.2,
          verbose=True,
          analysis_name= None, 
          results_dir="Results",
          force=False)


    # Validating inputs
    if(min_genes < 50 or min_genes > 2900):
        print("Variable min_genes must be between 50 and 2900 genes")
        return
    
    if(max_genes < 100 or max_genes > 3000):
        print("Variable max_genes must be between 100 and 3000 genes")
        return
    
    if(p_cut < 0.00001 or p_cut > 1):
        print("Variable p_cut must be between 0.00001 and 1")
        return
    



    # Preparing files and workspace
    ## Create a results directory
    
    if not os.path.exists(dirName): #cria diretorios se n existem
        os.makedirs("Results/Tables") 
        os.makedirs("Results/Heatmaps") 

        
    else:    
        if(force) # se diretorio existe e eh pra forcar, faz nada e bola pra frente
        else:     # se diretorio existe e forca eh falso, printa que ja existe o diretorio e aborta a funcao
            print("Stopping analysis: ", results_dir, " already exists! Use force=True to overwrite.")
            return
 
    

    
    ## Get information from all unique terms
    term2gene , path_desc , merge_p  = utils.read_gmt_hier(gmt_file)    # alterei funcao pra retornar o merge_p, ta retornando no formato de vetor mas talvez tenha q mudar para dataframe ou serie
    
    
    
    if( isinstance(file_in , str)):
        deg_list = pd.read_csv(file_in, header = TRUE, sep = "\t")
        if(analysis_name is None):
            analysis_name =   os.path.splitext(os.path.basename(file_in) )[0] # pega o basename e tira a extensao
     
    elif(isinstance(file_in , pd.DataFrame)):
        deg_list = file_in
    

    if(analysis_name is None):
        analysis_name = "PEBBA_analysis"
    
